In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "Fold")

22/03/02 01:48:08 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 01:48:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 01:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 01:48:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/02 01:48:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/02 01:48:10 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [8]:
# fold is useful for aggregate
# fold has initial value to start with where as reduce will  take first value as reduce

def add(acc, value):
    result = acc + value
    print("acc", acc, "value", value, "output", result)
    return result



In [9]:
rdd = sc.parallelize([10,20,30,40,50])

In [10]:
# fold with aggregate with start value 0
# fold is action method
# fold works with each partition first, calculate add function on each partition
# + it will apply result of all paritions into again another folder
# return value of add is passed as input with next number in seq

# after processing data from partition 0, it got result 150
# then it will apply add function across partition result  acc 0 value 150
rdd.fold (0, add) # output shall be 150 = 10 + 20 + 30 + 40 + 50

acc 0 value 150 output 150


acc 0 value 10 output 10
acc 10 value 20 output 30
acc 30 value 30 output 60
acc 60 value 40 output 100
acc 100 value 50 output 150


150

In [11]:
rdd = sc.parallelize([10,20,30,40,50], 2)
rdd.glom().collect()

[[10, 20], [30, 40, 50]]

In [12]:
# part 0: add is applied on part 0 = 10 + 20 = 30
# part 1: add is applied on part 1 = 30 + 40 + 50 = 120
# now output data set include (30, 120) which is used as input again with add func
# with start value 0
# final result: add is applied on the result of part 0 , part 1 = 30 + 120 = 150
rdd.fold (0, add)

acc 0 value 30 output 30
acc 30 value 120 output 150


acc 0 value 10 output 10
acc 10 value 20 output 30
acc 0 value 30 output 30
acc 30 value 40 output 70
acc 70 value 50 output 120


150

In [15]:
# FoldByKey
# similar to fold, where as fold is applied on all the values in RDD in partition
# foldByKey is used against (Key,Value) paired rdd, key/value rdd
# fold work based on key

orders = [
    # symbol, qty
    ('INFY', 200),
    ('TSLA', 50),
    ('EMR', 20),
    ('INFY', 100),
    ('TSLA', 25)
]

def add(acc, value):
    output = acc + value
    print("acc", acc, "value", value, "output", output)
    return output

orderRdd = sc.parallelize(orders)
# fold by Key, return rdd
# When key appear first, it starts with 0, and value
# second appearance key, include previous output as input
orderRdd.foldByKey(0, add).collect()

acc 0 value 200 output 200
acc 0 value 50 output 50
acc 0 value 20 output 20
acc 200 value 100 output 300
acc 50 value 25 output 75


[('INFY', 300), ('TSLA', 75), ('EMR', 20)]